In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import date
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import re

<h1>Part 1 - S&P and portfolio web scraper</h1>
<h2>Introduction</h2>
<p>Diversification, S&P golden standard, model portfolio diversification based on S&P and recommend stocks</p>
<p>This script scraps data from the yahoo finance statistics page.</p>
<img src="sample.JPG">
<p>Two sets of data are in focus</p>
<ol>
    <li>The S&P index constituents.</li>
    <li>One's portfolio</li>
    </ol>

## Item 1 - S&P Data
<h3>Getting updated S&P tickers from slickcharts website</h3>

In [3]:
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}

#Getting updated tickers from slickcharts website
def scrap_SnP_tickers(url, tableclass):
    resp = requests.get(url, headers = headers)
    soup = BeautifulSoup(resp.text, "html.parser")
    table = soup.find(class_ = tableclass)

    table_head = table.find('thead')
    header_list = []
    ths = table_head.find_all('th')
    for th in ths:
        header_list.append(th.text.strip())

    table_body = table.find('tbody')

    rows = table_body.find_all('tr')
    sp_data = []
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        sp_data.append([ele for ele in cols if ele]) # Get rid of empty values

    sp_df = pd.DataFrame(np.array(sp_data))
    sp_df.columns = header_list
    sp_df = sp_df.drop('#', axis=1)
    sp_df['Symbol'].replace(regex={r'[\.]': '-'}, inplace=True) #tickers need to have - instead of . for proper search on yahoo
    tickers = sp_df['Symbol'].tolist()
    print(f'Number of S&P constituent tickers = {len(tickers)}')
    display(sp_df.head())
    return sp_df, tickers

sp_df, tickers = scrap_SnP_tickers(url='https://www.slickcharts.com/sp500', tableclass="table-responsive")

#Optional way is to input CSV file with list of tickers
'''Using a csv file to get tickers
input_tickers = pd.read_csv('s&p_constituents.csv')
tickers = input_tickers['Symbol'].tolist()'''

Number of S&P constituent tickers = 503


,Company,Symbol,Weight,Price,Chg,% Chg
0,Apple Inc.,AAPL,6.013437,129.98,0.37,(0.29%)
1,Microsoft Corporation,MSFT,5.574288,240.25,-0.76,(-0.32%)
2,Amazon.com Inc.,AMZN,2.318124,83.98,-0.20,(-0.24%)
3,Berkshire Hathaway Inc. Class B,BRK-B,1.7277,309.35,0.29,(0.09%)
4,Alphabet Inc. Class A,GOOGL,1.639187,88.57,0.12,(0.14%)


<h3>Retrieving headers of yahoo finance stats page (Optional)</h3>

In [4]:
def get_yf_headers(ticker_sample): #ticker_sample is any ticker(str) for the purpose of retrieving the headers
    #Getting headers
    url = f'https://finance.yahoo.com/quote/{ticker_sample}/key-statistics?p={ticker_sample}'
    resp = requests.get(url, headers = headers)
    print(f'Using {ticker_sample} for headers, status - {resp.status_code}')
    soup = BeautifulSoup(resp.text, "html.parser")

    titles = ['Ticker']
    rows = soup.find_all("tr")
    for row in rows:
        cols = row.find_all("td")
        ele = cols[0].text.strip() 
        titles.append(ele)

    metrics_df = pd.DataFrame({"Metrics":titles})
    metrics_df['Metrics'].replace(regex={r'[0-9]$': ''}, inplace = True) #removes the annotations appearing at the end of rows
    metrics_df.iloc[24:29,0].replace(regex={r'(\(.+\))': ''}, inplace = True) #remove the dates under rows 24-28
    metrics = metrics_df['Metrics'].tolist()
    print(f'Extraction of headers complete! Total metric columns = {len(metrics)}')
    return metrics

metrics = get_yf_headers('AAPL')

Using AAPL for headers, status - 200
Extraction of headers complete! Total metric columns = 61


<h3>Renaming the headers manually to indicate units clearly</h3>

In [14]:
#temp = metrics.copy()

metrics = ['Ticker',
 'Market Cap (B)',
 'Enterprise Value (B)',
 'Trailing P/E',
 'Forward P/E',
 'PEG Ratio (5 yr expected)',
 'Price/Sales (ttm)',
 'Price/Book (mrq)',
 'Enterprise Value/Revenue',
 'Enterprise Value/EBITDA',
 'Beta (5Y Monthly)',
 '52 Week Change (%)',
 'S&P500 52-Week Change (%)',
 '52 Week High',
 '52 Week Low',
 '50-Day Moving Average',
 '200-Day Moving Average',
 'Avg Vol 3 month (M)',
 'Avg Vol 10 day (M)',
 'Shares Outstanding',
 'Implied Shares Outstanding',
 'Float',
 '% Held by Insiders',
 '% Held by Institutions',
 'Shares Short (M)',
 'Short Ratio (M)',
 'Short % of Float',
 'Short % of Shares Outstanding',
 'Shares Short',
 'Forward Annual Dividend Rate',
 'Forward Annual Dividend Yield (%)',
 'Trailing Annual Dividend Rate',
 'Trailing Annual Dividend Yield (%)',
 '5 Year Average Dividend Yield',
 'Payout Ratio (%)',
 'Dividend Date',
 'Ex-Dividend Date',
 'Last Split Factor (x:1)',
 'Last Split Date',
 'Fiscal Year Ends',
 'Most Recent Quarter (mrq)',
 'Profit Margin (%)',
 'Operating Margin (ttm) (%)',
 'Return on Assets (ttm) (%)',
 'Return on Equity (ttm) (%)',
 'Revenue (ttm) (B)',
 'Revenue Per Share (ttm)',
 'Quarterly Revenue Growth (yoy) (%)',
 'Gross Profit (ttm) (B)',
 'EBITDA (B)',
 'Net Income Avi to Common (ttm) (B)',
 'Diluted EPS (ttm)',
 'Quarterly Earnings Growth (yoy) (%)',
 'Total Cash (mrq) (B)',
 'Total Cash Per Share (mrq)',
 'Total Debt (mrq) (B)',
 'Total Debt/Equity (mrq)',
 'Current Ratio (mrq)',
 'Book Value Per Share (mrq)',
 'Operating Cash Flow (ttm) (B)',
 'Levered Free Cash Flow (ttm) (B)']

#for visually checking if the list referencing is correct
#test = pd.DataFrame({'Old':temp, 'new':metrics})
#test.iloc[0:60,:]

### Scraping and casting ticker stats into 2D list

In [6]:
def scrap_ticker(batch, start, stop):  
    #batch is a list of tickers
    #start should be 0
    #stop should be length of the list.
    
    start_time = datetime.now()
    if all_data:
        batch_number = max(all_data.keys())+1
        print(f'Batch number: {batch_number}')
    else:
        batch_number = 1
    
    batch_n_count = 0
    batch_data = []
    batch_interval = 50
    current_try = 1
    missed_tickers=[]
    
    for n in range(start, stop):
        url = f'https://finance.yahoo.com/quote/{batch[n]}/key-statistics?p={batch[n]}'
        resp = requests.get(url, headers = headers)
        print(f'{batch[n]} status - {resp.status_code}, {n+1}/{len(batch)}')
        soup = BeautifulSoup(resp.text, "html.parser")
        
        title = soup.find("h1")
        data= [title.text]
        
        rows = soup.find_all("tr")
        for row in rows:
            cols = row.find_all("td")
            ele = cols[1].text.strip()
            data.append(ele)
        
        if len(data)!=len(metrics) or data[1] == 'N/A':
            if len(data)!=len(metrics):
                print(f'length error({len(data)} instead of {len(metrics)}) in {batch[n]}')
            elif data[1] == 'N/A':
                print(f'N/A found in Marketcap of {batch[n]}')
            missed_tickers.append(batch[n])
            time.sleep(60*current_try)
            current_try +=1
        else:
            batch_data.append(data)
            print(f'{batch[n]} complete!')
            batch_n_count +=1
            time.sleep(5)
        if batch_n_count == batch_interval:
            all_data[batch_number] = pd.DataFrame(np.array(batch_data))
            print(f'Length of info extracted is {len(batch_data)} in batch {batch_number}')
            batch_n_count = 0
            batch_number +=1
            batch_data = []
            time.sleep(20)
    #the final appending for last batch with n smaller than 50
    all_data[batch_number] = pd.DataFrame(np.array(batch_data))
    print(f'Length of info extracted is {len(batch_data)} in batch {batch_number}')
    end_time = datetime.now()
    print('Elapsed time was', (end_time - start_time))
    return all_data, missed_tickers


all_data={}

all_data, missed_tickers = scrap_ticker(tickers, 0, len(tickers))

AAPL status - 200, 1/503
AAPL complete!
MSFT status - 200, 2/503
MSFT complete!
AMZN status - 200, 3/503
AMZN complete!
BRK-B status - 200, 4/503
BRK-B complete!
GOOGL status - 200, 5/503
GOOGL complete!
UNH status - 200, 6/503
UNH complete!
GOOG status - 200, 7/503
GOOG complete!
JNJ status - 200, 8/503
JNJ complete!
XOM status - 200, 9/503
XOM complete!
JPM status - 200, 10/503
JPM complete!
NVDA status - 200, 11/503
NVDA complete!
PG status - 200, 12/503
PG complete!
V status - 200, 13/503
V complete!
HD status - 200, 14/503
HD complete!
TSLA status - 200, 15/503
TSLA complete!
CVX status - 200, 16/503
CVX complete!
MA status - 200, 17/503
MA complete!
LLY status - 200, 18/503
LLY complete!
PFE status - 200, 19/503
PFE complete!
ABBV status - 200, 20/503
ABBV complete!
MRK status - 200, 21/503
MRK complete!
META status - 200, 22/503
META complete!
PEP status - 200, 23/503
PEP complete!
KO status - 200, 24/503
KO complete!
BAC status - 200, 25/503
BAC complete!
AVGO status - 200, 26/

ECL status - 200, 204/503
ECL complete!
DOW status - 200, 205/503
DOW complete!
KMI status - 200, 206/503
KMI complete!
ENPH status - 200, 207/503
ENPH complete!
COF status - 200, 208/503
COF complete!
HAL status - 200, 209/503
HAL complete!
CARR status - 200, 210/503
CARR complete!
NUE status - 200, 211/503
NUE complete!
HLT status - 200, 212/503
HLT complete!
PCAR status - 200, 213/503
PCAR complete!
ED status - 200, 214/503
ED complete!
DD status - 200, 215/503
DD complete!
HSY status - 200, 216/503
HSY complete!
IDXX status - 200, 217/503
IDXX complete!
CMI status - 200, 218/503
CMI complete!
AMP status - 200, 219/503
AMP complete!
BK status - 200, 220/503
BK complete!
OTIS status - 200, 221/503
OTIS complete!
MTD status - 200, 222/503
MTD complete!
KHC status - 200, 223/503
KHC complete!
TDG status - 200, 224/503
TDG complete!
EA status - 200, 225/503
EA complete!
FTNT status - 200, 226/503
FTNT complete!
AME status - 200, 227/503
AME complete!
ILMN status - 200, 228/503
ILMN comp

VTRS status - 200, 402/503
VTRS complete!
RE status - 200, 403/503
RE complete!
HRL status - 200, 404/503
HRL complete!
IPG status - 200, 405/503
IPG complete!
TECH status - 200, 406/503
TECH complete!
EXPE status - 200, 407/503
EXPE complete!
JKHY status - 200, 408/503
JKHY complete!
KIM status - 200, 409/503
KIM complete!
NDSN status - 200, 410/503
NDSN complete!
LW status - 200, 411/503
LW complete!
PTC status - 200, 412/503
PTC complete!
TRMB status - 200, 413/503
TRMB complete!
IP status - 200, 414/503
IP complete!
DPZ status - 200, 415/503
DPZ complete!
UAL status - 200, 416/503
UAL complete!
EQT status - 200, 417/503
EQT complete!
SNA status - 200, 418/503
SNA complete!
GEN status - 200, 419/503
GEN complete!
POOL status - 200, 420/503
POOL complete!
PKG status - 200, 421/503
PKG complete!
CPT status - 200, 422/503
CPT complete!
BF-B status - 200, 423/503
BF-B complete!
UDR status - 200, 424/503
UDR complete!
TFX status - 200, 425/503
TFX complete!
MTCH status - 200, 426/503
MTC

In [7]:
#extracting tickers that had errors/incomplete info
count = 0
while missed_tickers:
    temp = missed_tickers.copy()
    all_data, missed_tickers = scrap_ticker(temp, 0, len(temp))
    count +=1
print('Ran', count ,'times')

Batch number: 12
WBD status - 200, 1/1
WBD complete!
Length of info extracted is 1 in batch 12
Elapsed time was 0:00:05.895789 s
Ran 1 times


<h3>Casting the dataframe and cleaning all the data</h3>

In [15]:
def clean_df(all_data, metrics):
    #Casting the dataframe
    frames = [all_data[x] for x in all_data]
    all_data_df = pd.concat(frames)
    all_data_df.columns = metrics
    all_data_tickers = all_data_df['Ticker']
    print('Casting data, index and columns to build the dataframe')
    display(all_data_df.head())
    
    #Reformatting large sums (Billion, million, thousand) and removing (%,) values
    def _num_reformat(x):
        x = re.sub("[,]", "", x)
        if x[-1] == 'T':
            x = round(float(x[:-1])*1000,2)
        elif x[-1] == 'B':
            x = round(float(x[:-1]),2)
        elif x[-1] == 'M':
            x = round(float(x[:-1])*0.001,2)
        elif x[-1] == 'k':
            x = round(float(x[:-1])*0.000001,2)
        elif x[-1] == '%':
            x = round(float(x[:-1]),0)             
        elif x == "N/A":
            x = 0
        return x

    cleaned_df = all_data_df.iloc[:,1:].applymap(_num_reformat)
    cleaned_df.insert(0, 'Ticker', all_data_tickers)
    print('Reformatting large sums (Billion, million, thousand) and removing (%,) values')
    display(cleaned_df.head())
    
    #Handling the stock split factor column
    def _stocksplits(x):
        if type(x)==str:
            x = x.split(':')
            return round(int(x[0])/ int(x[1]),2)
        else:
            return x
    
    cleaned_df['Last Split Factor (x:1)'] = cleaned_df['Last Split Factor (x:1)'].apply(_stocksplits)
    print('Handling the stock split factor column')
    display(cleaned_df.iloc[:,[0,37]].head())
    
    dates = cleaned_df.iloc[:,[35, 36, 38, 39, 40]]
    
    #Converting dates to datetime format at the end of the table
    def _date_conversion(x):
        if x == 0:
            return np.nan
        else:
            x = str(x)
            return datetime.strptime(x, '%b %d %Y').date()

    dates = dates.applymap(_date_conversion)
    dates = dates.replace({0:np.nan})

    final_df = cleaned_df.drop(cleaned_df.columns[[0, 35, 36, 38, 39, 40]], axis = 1).astype('float64')
    final_df = pd.concat([final_df, dates], axis = 1)
    final_df.insert(0, 'Ticker', all_data_tickers)
    print('Converting dates to datetime format at the end of the table')
    display(final_df.head())
    
    #return the completed df
    return final_df

final_df = clean_df(all_data, metrics)

Casting data, index and columns to build the dataframe


,Ticker,Market Cap (B),Enterprise Value (B),Trailing P/E,Forward P/E,PEG Ratio (5 yr expected),Price/Sales (ttm),Price/Book (mrq),Enterprise Value/Revenue,Enterprise Value/EBITDA,...,Diluted EPS (ttm),Quarterly Earnings Growth (yoy) (%),Total Cash (mrq) (B),Total Cash Per Share (mrq),Total Debt (mrq) (B),Total Debt/Equity (mrq),Current Ratio (mrq),Book Value Per Share (mrq),Operating Cash Flow (ttm) (B),Levered Free Cash Flow (ttm) (B)
0,Apple Inc. (AAPL),2.07T,2.14T,21.28,21.28,2.53,5.38,40.82,5.43,16.08,...,6.11,0.80%,48.3B,3.04,132.48B,261.45,0.88,3.18,122.15B,90.22B
1,Microsoft Corporation (MSFT),1.80T,1.75T,25.94,25.32,2.13,8.92,10.35,8.62,17.35,...,9.29,-14.40%,107.24B,14.39,77.14B,44.44,1.84,23.28,87.69B,46.16B
2,"Amazon.com, Inc. (AMZN)",858.78B,928.36B,76.49,41.49,4.68,1.71,6.25,1.85,18.31,...,1.13,-9.00%,58.66B,5.75,164.33B,119.53,0.94,13.48,39.67B,-12.69B
3,Berkshire Hathaway Inc. (BRK-B),669.47B,759.51B,N/A,21.05,N/A,2.64,1.46,2.99,N/A,...,5.21,N/A,108.96B,"74,383.3",116.5B,25.12,1.46,"310,560.19",34.82B,-23.77B
4,Alphabet Inc. (GOOGL),1.15T,1.06T,17.58,16.56,1.25,4.16,4.51,3.76,11.27,...,5.17,-26.50%,116.26B,8.98,29.34B,11.57,2.52,19.55,92.81B,48.26B


Reformatting large sums (Billion, million, thousand) and removing (%,) values


,Ticker,Market Cap (B),Enterprise Value (B),Trailing P/E,Forward P/E,PEG Ratio (5 yr expected),Price/Sales (ttm),Price/Book (mrq),Enterprise Value/Revenue,Enterprise Value/EBITDA,...,Diluted EPS (ttm),Quarterly Earnings Growth (yoy) (%),Total Cash (mrq) (B),Total Cash Per Share (mrq),Total Debt (mrq) (B),Total Debt/Equity (mrq),Current Ratio (mrq),Book Value Per Share (mrq),Operating Cash Flow (ttm) (B),Levered Free Cash Flow (ttm) (B)
0,Apple Inc. (AAPL),2070.00,2140.00,21.28,21.28,2.53,5.38,40.82,5.43,16.08,...,6.11,1.0,48.30,3.04,132.48,261.45,0.88,3.18,122.15,90.22
1,Microsoft Corporation (MSFT),1800.00,1750.00,25.94,25.32,2.13,8.92,10.35,8.62,17.35,...,9.29,-14.0,107.24,14.39,77.14,44.44,1.84,23.28,87.69,46.16
2,"Amazon.com, Inc. (AMZN)",858.78,928.36,76.49,41.49,4.68,1.71,6.25,1.85,18.31,...,1.13,-9.0,58.66,5.75,164.33,119.53,0.94,13.48,39.67,-12.69
3,Berkshire Hathaway Inc. (BRK-B),669.47,759.51,0,21.05,0,2.64,1.46,2.99,0,...,5.21,0.0,108.96,74383.3,116.50,25.12,1.46,310560.19,34.82,-23.77
4,Alphabet Inc. (GOOGL),1150.00,1060.00,17.58,16.56,1.25,4.16,4.51,3.76,11.27,...,5.17,-26.0,116.26,8.98,29.34,11.57,2.52,19.55,92.81,48.26


Handling the stock split factor column


,Ticker,Last Split Factor (x:1)
0,Apple Inc. (AAPL),4.0
1,Microsoft Corporation (MSFT),2.0
2,"Amazon.com, Inc. (AMZN)",20.0
3,Berkshire Hathaway Inc. (BRK-B),50.0
4,Alphabet Inc. (GOOGL),20.0


Converting dates to datetime format at the end of the table


,Ticker,Market Cap (B),Enterprise Value (B),Trailing P/E,Forward P/E,PEG Ratio (5 yr expected),Price/Sales (ttm),Price/Book (mrq),Enterprise Value/Revenue,Enterprise Value/EBITDA,...,Total Debt/Equity (mrq),Current Ratio (mrq),Book Value Per Share (mrq),Operating Cash Flow (ttm) (B),Levered Free Cash Flow (ttm) (B),Dividend Date,Ex-Dividend Date,Last Split Date,Fiscal Year Ends,Most Recent Quarter (mrq)
0,Apple Inc. (AAPL),2070.00,2140.00,21.28,21.28,2.53,5.38,40.82,5.43,16.08,...,261.45,0.88,3.18,122.15,90.22,2022-11-09,2022-11-03,2020-08-30,2022-09-23,2022-09-23
1,Microsoft Corporation (MSFT),1800.00,1750.00,25.94,25.32,2.13,8.92,10.35,8.62,17.35,...,44.44,1.84,23.28,87.69,46.16,2023-03-08,2023-02-14,2003-02-17,2022-06-29,2022-09-29
2,"Amazon.com, Inc. (AMZN)",858.78,928.36,76.49,41.49,4.68,1.71,6.25,1.85,18.31,...,119.53,0.94,13.48,39.67,-12.69,NaN,NaN,2022-06-05,2021-12-30,2022-09-29
3,Berkshire Hathaway Inc. (BRK-B),669.47,759.51,0.00,21.05,0.00,2.64,1.46,2.99,0.00,...,25.12,1.46,310560.19,34.82,-23.77,NaN,NaN,2010-01-20,2021-12-30,2022-09-29
4,Alphabet Inc. (GOOGL),1150.00,1060.00,17.58,16.56,1.25,4.16,4.51,3.76,11.27,...,11.57,2.52,19.55,92.81,48.26,NaN,NaN,2022-07-17,2021-12-30,2022-09-29


### Exporting to csv file for backup and further analysis

In [16]:
current_datetime = date.today().isoformat()
final_df.to_csv(f'data/s&p_{current_datetime}.csv')
print(f'Saved to file: s&p_{current_datetime}.csv')

Saved to file: s&p_2022-12-31.csv


## Item 2 - Personal portfolio data

In [ ]:
input_tickers = pd.read_csv('portfolio_tickers.csv') #input file with a list of portfolio tickers
tickers = input_tickers['Tickers'].tolist()

all_data={}

all_data, missed_tickers = scrap_ticker(tickers, 0, len(tickers))

#extracting tickers that had errors/incomplete info
count = 0
while missed_tickers:
    temp = missed_tickers.copy()
    all_data, missed_tickers = scrap_ticker(temp, 0, len(temp))
    count +=1
print('Ran', count ,'times')

final_df = clean_df(all_data, metrics)

In [ ]:
current_datetime = date.today().isoformat()
final_df.to_csv(f'data/portfolio_{current_datetime}.csv')
print(f'Saved to file: portfolio_{current_datetime}.csv')